In [41]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

API_PATH=r"/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/config/api_keys.env"
KEY_NAME="NVIDIA_API_KEY"
SYSTEM_PROMPT_PATH=r"/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/config/system_prompt.txt"

MODEL_BASE_URL="https://integrate.api.nvidia.com/v1"
MODEL_TYPE="nvidia/llama-3.3-nemotron-super-49b-v1"

def load_api_key(api_path: str, key_name: str) -> str:
    """Load API key from environment file"""
    load_dotenv(api_path)
    api_key = os.getenv(key_name)
    if not api_key:
        raise ValueError(f"Missing {key_name} in environment")
    return api_key

def load_system_prompt(system_prompt_path: str) -> str:
    """Load system prompt from file"""
    with open(system_prompt_path, "r", encoding="utf-8") as f:
        return f.read()

def load_asset_input_data(file_path: str) -> Dict[str, Any]:
    """Load and parse assets input data from file"""
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)
    
def run_sentiment_inference(asset_json_path: str) -> dict:
    """
    Run sentiment risk analysis on a given JSON file of asset data using NVIDIA's LLM API.

    Args:
        asset_json_path (str): Path to the JSON file with asset names and associated text data.

    Returns:
        dict: Parsed JSON result from the LLM containing risk assessments.
    """

    # Load API key
    api_key = load_api_key(API_PATH, KEY_NAME)
    
    # Load system prompt
    system_prompt = load_system_prompt(SYSTEM_PROMPT_PATH)

    # Load asset data
    user_input = user_input = json.dumps(load_asset_input_data(asset_json_path))

    # Initialize OpenAI client
    client = OpenAI(
        base_url=MODEL_BASE_URL,
        api_key=api_key
    )

    # Create chat completion
    completion = client.chat.completions.create(
        model=MODEL_TYPE,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0.6,
        top_p=0.95,
        max_tokens=4096,
        frequency_penalty=0,
        presence_penalty=0,
        stream=True
    )

    # Stream and capture response
    output = ""
    for chunk in completion:
        content = chunk.choices[0].delta.content
        if content is not None:
            print(content, end="")
            output += content

    # Parse and return JSON
    try:
        result_json = json.loads(output)
        return result_json
    except json.JSONDecodeError as e:
        raise RuntimeError("Failed to parse LLM output as JSON.") from e

In [40]:
if __name__ == "__main__":
    result = run_sentiment_inference("/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/data/assets_output.json")
    with open("/Users/akramchakrouni/eth-wallet-risk-analysis/sentiment_analysis/data/output2.json", "w", encoding="utf-8") as f:
        json.dump(result, f, indent=2)

[
  {
    "asset_name": "LINK",
    "risk_level": "MEDIUM",
    "reason": "Mixed signals: Positive integrations (PayPal, Cardano) and price performance (ADA & LINK leading BTC) are balanced by unrelated negative news (Trump-linked AVAX/MNT purchase with significant losses) and a potentially misleading or outdated article about Donald Trump's Ethereum/Chainlink purchase, which lacks clear relevance or recent verification.",
    "source": [
      "https://cryptopanic.com/news/20938494/PayPal-Adds-Solana-and-Chainlink-to-its-Crypto-Services?mtm_campaign=API-OFA",
      "https://cryptopanic.com/news/20894182/Hoskinson-Reveals-Cardanos-Push-to-Integrate-with-Dogecoin-Chainlink-Bitcoin?mtm_campaign=API-OFA",
      "https://cryptopanic.com/news/20783012/ADA-and-LINK-Steal-the-Show-as-BTC-Eyes-90K-Market-Watch?mtm_campaign=API-OFA",
      "https://cryptopanic.com/news/20831805/Trump-backed-World-Liberty-Financial-buys-AVAX-and-MNT-for-4M-as-total-portfolio-losses-reach-118M?mtm_campaign=API-OF